In [1]:
# import all modules needed
import models.eda as eda

import pandas as pd
import numpy as np
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


In [2]:
# df_client and df_invoice are being loaded from the data folder
client = eda.load_data("data/train/client_train.csv")
invoice = eda.load_data("data/train/invoice_train.csv")
df = eda.feature_change(client, invoice)

/var/folders/zj/y95dm4bn0jz_s678nqhm92b80000gn/T/ipykernel_21011/3719668010.py:3: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  invoice = eda.load_data("data/train/invoice_train.csv")


In [3]:
df.head(20)

,district,client_id,client_catg,region,creation_date,target,region_group,coop_time,invoice_date,tarif_type,...,counter_statue,counter_code,reading_remarque,counter_coefficient,months_number,counter_type,invoice_month,invoice_year,is_weekday,total_consumption
0,60,0,11,101,1994-12-31,0.0,200,288,2014-03-24,11,...,0,203,8,1,4,1,3,2014,0.0,82
1,60,0,11,101,1994-12-31,0.0,200,288,2013-03-29,11,...,0,203,6,1,4,1,3,2013,0.0,1384
2,60,0,11,101,1994-12-31,0.0,200,288,2015-03-23,11,...,0,203,8,1,4,1,3,2015,0.0,123
3,60,0,11,101,1994-12-31,0.0,200,288,2015-07-13,11,...,0,207,8,1,4,1,7,2015,0.0,102
4,60,0,11,101,1994-12-31,0.0,200,288,2016-11-17,11,...,0,207,9,1,12,1,11,2016,0.0,572
5,60,0,11,101,1994-12-31,0.0,200,288,2017-07-17,11,...,0,207,9,1,8,1,7,2017,0.0,314
6,60,0,11,101,1994-12-31,0.0,200,288,2018-12-07,11,...,0,207,9,1,12,1,12,2018,0.0,541
7,60,0,11,101,1994-12-31,0.0,200,288,2019-03-19,11,...,0,207,9,1,8,1,3,2019,0.0,585
8,60,0,11,101,1994-12-31,0.0,200,288,2011-07-22,11,...,0,203,9,1,4,1,7,2011,0.0,1386
9,60,0,11,101,1994-12-31,0.0,200,288,2011-11-22,11,...,0,203,6,1,4,1,11,2011,0.0,1082


### Baseline model
- our very simple baseline model assumes every client from district 51 is fraudulent. 

In [4]:
def baseline_model(X):
    y_pred = [0 if cat != 51 else 1 for cat in X.client_catg]
    return y_pred

# in district 51 the most fraudulents appear:
pd.crosstab(df['target'], df['client_catg'], normalize='columns')*100

client_catg,11,12,51
target,,,
0.0,92.391486,94.413284,79.044952
1.0,7.608514,5.586716,20.955048


In [36]:
# Select the features to use for modeling
feature_cols = ['district','client_catg', 'region_group',
       'tarif_type', 'counter_number',
       'counter_statue', 'counter_code', 'reading_remarque',
       'counter_coefficient', 'months_number', 'counter_type',
       'total_consumption', 'invoice_year']
X_train, X_test, y_train, y_test = train_test_split(df[feature_cols], df['target'], test_size=0.3, random_state=42)
y_pred = baseline_model(X_test)


In [37]:
print('F1-score:', precision_score(y_test, y_pred))
print('ROC AUC:', roc_auc_score(y_test, y_pred))

F1-score: 0.21303856704857188
ROC AUC: 0.5212540577905068


## Preprocessing

In [ ]:
# balance data

In [ ]:
# scale data

## A model with better performance? 

In [19]:
# Calculate the mean total consumption per client and per year of invoice
# df_clean["mean_consumption"] = df_clean.groupby(['client_id', 'invoice_year'])['total_consumption'].transform("mean")


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df[feature_cols], df['target'], test_size=0.3, random_state=42)

# Create and train the logistic regression model
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

# Predict the target variable for the test set
y_pred = lr_model.predict(X_test)

# Evaluate the performance of the model
print('F1-score:', precision_score(y_test, y_pred)) # <-- use this one for imbalanced data or F2 or F3
print('ROC AUC:', roc_auc_score(y_test, y_pred))

F1-score: 0.0
ROC AUC: 0.5


/Users/tom/Documents/neuefische/ds-fraud-detection/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
